In [7]:
%env CYPHER=/home/jupyter/neo4j-community-4.3.3/bin/cypher-shell
%env USER=neo4j
%env PW=neo4j

env: CYPHER=/home/jupyter/neo4j-community-4.3.3/bin/cypher-shell
env: USER=neo4j
env: PW=neo4j


In [8]:
CONNECT="$CYPHER -u $USER -p $PW"

In [3]:
!$CONNECT "MATCH (n) DETACH DELETE n"

0 rows
ready to start consuming query after 145 ms, results consumed after another 0 ms


In [11]:
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain

COUNT(l)
5835
COUNT(a)
42
COUNT(n)
41
COUNT(h)
4633
COUNT(u)
55917
COUNT(r)
62976


In [12]:
!{CONNECT} "MATCH (n) RETURN count(n)"

+----------+
| count(n) |
+----------+
| 129444   |
+----------+

1 row
ready to start consuming query after 10 ms, results consumed after another 0 ms


In [10]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(n)"

+----------------------+
| labels(n) | count(n) |
+----------------------+
+----------------------+

0 rows
ready to start consuming query after 104 ms, results consumed after another 7 ms


In [11]:
!{CONNECT} "MATCH (n) RETURN n LIMIT 10"

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| n                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Listing {bedrooms: 1, listing_id: "72635", availability_365: 240, price: 300.0, accommodates: 6, name: "3 Private Bedrooms, SW Austin", property_type: "House", bathrooms: 2})                                      |
| (:Listing {bedrooms: 1, cleaning_fee: 75.0, weekly_price: 600.0, listing_id: "5386323", availability_365: 364, price: 99.0, ac

In [21]:
!{CONNECT} "MATCH ()-[r]->() RETURN type(r) LIMIT 10"

+-------------------+
| type(r)           |
+-------------------+
| "IN_NEIGHBORHOOD" |
| "HAS"             |
| "HAS"             |
| "HAS"             |
| "HAS"             |
| "HAS"             |
| "HAS"             |
| "HAS"             |
| "HAS"             |
| "HAS"             |
+-------------------+

10 rows
ready to start consuming query after 1 ms, results consumed after another 1 ms


Q1. How many hosts are located in "Austin, Texas, United States"?

In [32]:
!{CONNECT} "MATCH (m:Host {{location: 'Austin, Texas, United States'}}) RETURN count(m) LIMIT 10"

+----------+
| count(m) |
+----------+
| 3774     |
+----------+

1 row
ready to start consuming query after 38 ms, results consumed after another 19 ms


Q2. Which listings does host_id = "4641823" have? Return the listing name, property_type,
price, and availability_365 sorted by price. Limit the results to 10.

In [13]:
!{CONNECT} "MATCH (p1:Host {{host_id:'4641823'}})-[:HOSTS]->(m) RETURN m.name, m.property_type , m.price, m.availability_365 ORDER BY m.price LIMIT 10"

+----------------------------------------------------------------------------------------+
| m.name                                | m.property_type | m.price | m.availability_365 |
+----------------------------------------------------------------------------------------+
| "1BR Convenient Austin Condo "        | "Apartment"     | 93.0    | 354                |
| "1BR Inviting Downtown Condo, Pool"   | "Apartment"     | 99.0    | 335                |
| "2BR/1.5BA Charming House Off SoCo"   | "House"         | 110.0   | 357                |
| "2BR Prime East-Side Downtown"        | "House"         | 121.0   | 341                |
| "1BR SoCo Treehouse Studio"           | "House"         | 129.0   | 327                |
| "1BR/1.5BA East 6th, Colorful 2Story" | "Apartment"     | 134.0   | 344                |
| "3BR Prestigious Home Travis Heights" | "House"         | 138.0   | 0                  |
| "1BR/1.5BA Perfectly Located Casita"  | "House"         | 140.0   | 351                |

Q3. Which users wrote a review for listing_id = "5293632"? Return the user’s id and name
sorted alphabetically by name. Limit the results to 10.

In [29]:
!{CONNECT} "MATCH (p1:User)-[:WROTE]->(m)-[:REVIEWS]->(l:Listing {{listing_id: '5293632'}}) return p1.name, p1.user_id ORDER BY p1.name LIMIT 10"

+--------------------------------+
| p1.name           | p1.user_id |
+--------------------------------+
| "Annie"           | "18286390" |
| "Carole"          | "30193020" |
| "Cory"            | "16497582" |
| "Craig And Trina" | "35022795" |
| "Dianne"          | "13281665" |
| "Hannah"          | "29601600" |
| "Jacob"           | "11940539" |
| "Jessie"          | "3213433"  |
| "Johannes"        | "41722221" |
| "Ju-Ju"           | "28480571" |
+--------------------------------+

10 rows
ready to start consuming query after 67 ms, results consumed after another 2 ms


Q4. Which users wrote a review for any listing which has the amenities "Washer" and "Dryer"?
Return the user’s id and name sorted alphabetically by name. Limit the results to 10.

In [82]:
!{CONNECT} "MATCH (p1:User)-[:WROTE]->(m)-[:REVIEWS]->(l)-[:HAS]->(a:Amenity {{name: 'Washer / Dryer'}})  return p1.name, p1.user_id, a.name ORDER BY p1.name LIMIT 10"

+-------------------------------------------+
| p1.name   | p1.user_id | a.name           |
+-------------------------------------------+
| "Andrew"  | "1627563"  | "Washer / Dryer" |
| "Bryan"   | "1631432"  | "Washer / Dryer" |
| "Cindy"   | "2784203"  | "Washer / Dryer" |
| "Henrik"  | "416470"   | "Washer / Dryer" |
| "Scott"   | "386269"   | "Washer / Dryer" |
| "Thaniya" | "430392"   | "Washer / Dryer" |
| "Warren"  | "316879"   | "Washer / Dryer" |
+-------------------------------------------+

7 rows
ready to start consuming query after 49 ms, results consumed after another 1 ms


Q5. Which listings have 3 bedrooms and are located in the Clarksville neighborhood? Return
the listing name, property_type, price, and availability_365 sorted by price. Limit the results
to 5.


In [71]:
!{CONNECT} "MATCH (l:Listing {{bedrooms: 3}}) -[IN_NEIGHBORHOOD]-> (n:Neighborhood  {{name: 'Clarksville'}}) return l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 5"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "3BR/2.5BA Exquisite Townhouse"       | "House"         | 222.0   | 358                |
| "3BR/2.5BA Tarrytown Duplex, Austin!" | "House"         | 249.0   | 336                |
| "Austin downtown hideaway"            | "House"         | 249.0   | 364                |
| "3BD Luxury Cottage by Lake Austin"   | "House"         | 290.0   | 309                |
| "Entire Adorable Downtown House"      | "House"         | 295.0   | 309                |
+----------------------------------------------------------------------------------------+

5 rows
ready to start consuming query after 49 ms, results consumed after another 2 ms


Q6. Which amenities are the most common? Return the name of the amenity and its frequency.
Sort the results by count in descending order. Limit the results to 5.

In [80]:
!{CONNECT} "MATCH (l:Listing) -[:HAS]-> (a:Amenity) return distinct a.name, count(a.name) as totals ORDER BY -totals LIMIT 5"

+-------------------------------------+
| a.name                     | totals |
+-------------------------------------+
| "Air Conditioning"         | 5615   |
| "Wireless Internet"        | 5479   |
| "Heating"                  | 5440   |
| "Kitchen"                  | 5400   |
| "Free Parking on Premises" | 5123   |
+-------------------------------------+

5 rows
ready to start consuming query after 34 ms, results consumed after another 116 ms


Q7. Which neighborhoods have the highest number of listings? Return the neighborhood’s
name and zip code (neighborhood_id) along with the number of listings they have sorted
by the number of listings in descending order. Limit the results to 5.

In [103]:
!{CONNECT} "MATCH (l:Listing) -[:IN_NEIGHBORHOOD]-> (n:Neighborhood) return distinct n.neighborhood_id as zipcode, count(n.neighborhood_id) as totals, n.name as name  ORDER BY -totals LIMIT 5"

+-------------------------------------+
| zipcode | totals | name             |
+-------------------------------------+
| "78704" | 1601   | NULL             |
| "78702" | 797    | NULL             |
| "78703" | 419    | "Clarksville"    |
| "78741" | 414    | "East Riverside" |
| "78745" | 328    | NULL             |
+-------------------------------------+

5 rows
ready to start consuming query after 28 ms, results consumed after another 9 ms
